<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spatial-Joining-to-Counties" data-toc-modified-id="Spatial-Joining-to-Counties-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Spatial Joining to Counties</a></span></li><li><span><a href="#Creating-a-map-with-Folium" data-toc-modified-id="Creating-a-map-with-Folium-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a map with Folium</a></span><ul class="toc-item"><li><span><a href="#A-Folium-map-requires:" data-toc-modified-id="A-Folium-map-requires:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>A Folium map requires:</a></span></li><li><span><a href="#Folium-Geodata-needs-these-attributes-:" data-toc-modified-id="Folium-Geodata-needs-these-attributes-:-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Folium Geodata needs these attributes :</a></span></li><li><span><a href="#Calculating-the-map-center" data-toc-modified-id="Calculating-the-map-center-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Calculating the map center</a></span></li><li><span><a href="#Creating-a-folium-map" data-toc-modified-id="Creating-a-folium-map-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Creating a folium map</a></span></li><li><span><a href="#Save-the-Folium-Map" data-toc-modified-id="Save-the-Folium-Map-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Save the Folium Map</a></span></li></ul></li><li><span><a href="#Creating-a-Histogram-Map-with-Folium" data-toc-modified-id="Creating-a-Histogram-Map-with-Folium-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Creating a Histogram Map with Folium</a></span><ul class="toc-item"><li><span><a href="#Geodata-for-Folium-Histogram-map" data-toc-modified-id="Geodata-for-Folium-Histogram-map-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Geodata for Folium Histogram map</a></span><ul class="toc-item"><li><span><a href="#Exercise:-How-can-we-check-our-geodata-frames-data?" data-toc-modified-id="Exercise:-How-can-we-check-our-geodata-frames-data?-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span><em>Exercise</em>: How can we check our geodata frames data?</a></span></li></ul></li><li><span><a href="#folium.Choropleth-requires-these-parameters:" data-toc-modified-id="folium.Choropleth-requires-these-parameters:-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>folium.Choropleth requires these parameters:</a></span></li></ul></li><li><span><a href="#Adding-tooltips" data-toc-modified-id="Adding-tooltips-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Adding tooltips</a></span></li><li><span><a href="#Congratulations!-You-are-done!!" data-toc-modified-id="Congratulations!-You-are-done!!-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Congratulations! You are done!!</a></span><ul class="toc-item"><li><span><a href="#Potential-Solution-#2" data-toc-modified-id="Potential-Solution-#2-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Potential Solution #2</a></span></li></ul></li></ul></div>

# Alby Pythons Folium Circus

## Spatial Joining to Counties

Your web browser can only handle so much content, so creating a 8,000 polygon map with our EnviroScreen data is probably not a good idea! Our solution is to aggregate the data into bigger geographies, in this case __counties__.

 We will begin by reading in our data, convinently, the California county's shapefile is provided for you under `/data/shapefiles/CA_Counties_TIGER2016.shp`

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

# load our shapefiles
eda_gdf = gpd.read_file("./data/shapefiles/CA_EDA_analysis.shp")
ca_counties_gdf = gpd.read_file("./data/shapefiles/CA_Counties_TIGER2016.shp")

# check the coordinate reference systems
print("the crs for eda_gdf is {}".format(eda_gdf.crs))
print("the crs for ca_counties_gdf is {}".format(ca_counties_gdf.crs))

Since the coordinate reference systems do not align, we have to convert one them using `gdf.to_crs`

In [ ]:
projected_ca_counties_gdf = ca_counties_gdf.to_crs({'init': 'epsg:3310'})

spatial_join = gpd.sjoin(eda_gdf, projected_ca_counties_gdf, how="left")
dfpivot = pd.pivot_table(spatial_join,index='GEOID',columns='EDA_index')
dfpivot.columns = dfpivot.columns.droplevel()
dfpolynew = ca_counties_gdf.merge(dfpivot, how='left',on='GEOID')
dfpolynew

## Creating a map with Folium

With the join in place we can now start mapping with Folium!

### A Folium map requires:    
    - A basemap (can choose from here)
    - A geographic center
    - Zoom level
    - The WGS_1984 (EPSG=4326) Coordinate Reference System (CRS)

### Folium Geodata needs these attributes :
    - Projected in the WGS_1984 (EPSG=4326) Coordinate Reference System (CRS)
    - A geometry column and/or Latitude/Longitude

### Calculating the map center  
__Why is a map center necessary?__  
__`matplotlib`'s__`plot` function only shows the data, while __`folium`__ shows the world, so we want to make sure our map is centered properly.

We will still be using our EDA data frame (`EDA_gdf`), but we want to be sure it is projected correctly and use the geographic center of the dataset as the center for our map.

 `our_subset.to_crs(epsg='4326')` reprojects our data.  
 
 `our_data.total_bounds` will then give us the boundaries of our data set as a pair of two `[x,y]` coordinates.
 

In [ ]:
ead_county = gpd.read_file("./data/shapefiles/Join_Output.shp")
ead_county_data = ead_county[['GEOID', 'Avg_EDA_in', 'geometry']]
ead_county_data.simplify(tolerance=100).to_file("./data/ead_county.geojson", driver="GeoJSON")
# smaller_data = gpd.read_file("./data/smaller.shp")
ca_crop_extent = ead_county.to_crs(epsg='4326')

# ead_county['geometry'] = ead_county['geom_gen']
map_bounds = ca_crop_extent.total_bounds
print(map_bounds)
# We get the average of the map_center for the x and y values
x_center = (map_bounds[0]+map_bounds[2])/2
y_center = (map_bounds[1]+map_bounds[3])/2

# We can print the values here to check
print(x_center,y_center)

### Creating a folium map

After getting the map center, we will then create our __`folium`__ map centered around those values.

Since our `ead_county` is a geodata frame, we can translate it to a `json()` and have it easily shown in __`folium`__. 


In [ ]:
# create our map and feed the location our center values
fmap = folium.Map(location=[y_center,x_center], 
                    tiles='cartodbpositron',
                    zoom_start=6,
                    control_scale=True)

# convert our geodataframe into geojson
ead_county_json = ead_county_data.simplify(tolerance=300).to_crs(epsg='4326').to_json()
polygons = folium.features.GeoJson(ead_county_json)

In [ ]:
# add the geojson polygons to the map
fmap.add_child(polygons)
# show the map
fmap

###  Save the Folium Map

In [ ]:
outfp = "results/choropleth_map.html"
fmap.save(outfp)

With our map visualized (except in Chrome browsers) we can now move onto the next step of creating a histogram!

## Creating a Histogram Map with Folium

Instead of converting our geodata frame into a GeoJSON, we will directly use a geodata frame to populate our histogram map. There are a few more requirements to address when creating a map in this fashion though.

### Geodata for Folium Histogram map
    - Requires a unique identifier for each record, usually called 'geoid'
    - Selecting a subset of the geodata frame improves performance

    - Projected in the WGS_1984 (EPSG=4326) Coordinate Reference System (CRS)
    - A geometry column and/or Latitude/Longitude

#### _Exercise_: How can we check our geodata frames data?

In [ ]:
# Perform your check in this cell #
ead_county.head()

Reading our data's head shows us that we already have a `geoid` column and a `geometry` column, so all we need to do is select a __subset of our data__ and re-assure that we are using the correct CRS `(EPSG=4326)`

In [ ]:
# create geoid
ead_county['geoid'] = ead_county.index.astype(str)

ead_county_data = ead_county[["geoid", "NAME",'Avg_EDA_in','geometry']]
# ead_county_data.rename(columns={'FID_1':'id'}, inplace=True)
print(ead_county_data.head())
# ead_county_data.to_crs({'init': 'epsg:4326'})
# ead_county_data_json = ead_county_data.simplify(tolerance=300).to_crs(epsg='4326').to_json()

# print(ead_county_data['Avg_EDA_in'])

Now we can use the `fmap` from earlier, but add the magical `folium.Choropleth()` function to create our colored map.

In [ ]:
fcmap = folium.Map(location=[y_center,x_center], 
                    tiles='cartodbpositron',
                    zoom_start=6,
                    control_scale=True)

In [ ]:
state_geo = "./data/ead_county.geojson"
state_data = ead_county_data

### folium.Choropleth requires these parameters:
  - geo_data = the geojson (ex. ead_county_data.geojson)
  - name = what the name of the webpage is (ex. California's Economically Distressed Area Index)
  - data = what the table data is (ex. ead_county_data)
  - columns = what columns to use (ex. 'geoid', 'Avg_EDA_in')
  - key_on = what to show on click (ex. 'feature.id')
  - legend_name = the name for the legend (ex. Average EDA index in County)

In [21]:
fmap_c = folium.Map(location=[y_center,x_center], 
                    tiles='cartodbpositron',
                    zoom_start=6,
                    control_scale=True)

folium.Choropleth(
    geo_data=ead_county_data,
    name="California's Economically Distressed Area Index",
    data=ead_county_data,
    columns=['geoid', 'Avg_EDA_in'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
#     smooth_factor=1.0,
#     threshold_scale=[1, 2, 3, 4, 5],
    legend_name= 'Average EDA index in County').add_to(fmap_c)

In [ ]:
fmap_c

In [ ]:
# save our map
outfp = "results/CA_EDA_map_fmap_c.html"
fmap_c.save(outfp)

## Adding tooltips

In [ ]:
for i in range(len(ead_county_data)):
    gs = folium.GeoJson(ead_county_data.iloc[i:i+1])
    label = '{}: {}% AVG EDA'.format(
        ead_county_data['NAME'][i], (ead_county_data['Avg_EDA_in'][i]))
    folium.Popup(label).add_to(gs)
    gs.add_to(fmap_c)

In [ ]:
outfp = "results/CA_EDA_map_fmap_c.html"
fmap_c.save(outfp)

## Congratulations! You are done!!

In [ ]:
fmap_c

### Potential Solution #2

In [22]:
folium.features.GeoJson(ead_county_data,  name='Labels',
               style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0,'line_opacity':0},
                tooltip=folium.features.GeoJsonTooltip(fields=['Avg_EDA_in'],
                                              aliases = ['Avg_EDA_in'],
                                              labels=True,
                                              sticky=False
                                             )
                       ).add_to(fmap_c)


In [23]:
outfp = "results/CA_EDA_map_fmap_c_final.html"
fmap_c.save(outfp)